In [6]:
# Your inputs
queries = [
    # "Is attention really all you need?",
    # "Are Benjamin, Antoine, Merve, and Jo best friends?",
    # "Long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long long"
    "hello world",
    "flag embedding",
]

In [2]:
from fastembed.text.colpali_model import ColpaliTextModel

model_f_i = ColpaliTextModel(model_name="akshayballal/colpali-v1.2-merged")
fastembed_q_embeddings = list(model_f_i.embed(queries))

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/243k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

model.onnx_data:   0%|          | 0.00/5.85G [00:00<?, ?B/s]

model.onnx:   0%|          | 0.00/2.08M [00:00<?, ?B/s]

In [5]:
fastembed_q_embeddings[0][0]

array([ 0.15808105, -0.020401  ,  0.10046387, -0.02319336,  0.04464722,
        0.03086853,  0.07092285,  0.03094482,  0.00642776, -0.04315186,
       -0.05871582, -0.12011719,  0.06945801, -0.06872559,  0.02192688,
        0.05203247,  0.04309082,  0.05557251,  0.14208984,  0.01930237,
        0.04016113, -0.03347778, -0.12988281,  0.01519775, -0.01997375,
        0.03387451, -0.07312012,  0.22009277, -0.04208374, -0.04373169,
        0.14050293,  0.05145264, -0.11676025, -0.08654785, -0.01660156,
        0.04003906, -0.06347656,  0.01754761, -0.00278473,  0.05606079,
        0.00063658, -0.07666016, -0.1842041 , -0.08789062,  0.09680176,
       -0.09796143,  0.22021484,  0.02334595,  0.06124878,  0.01942444,
       -0.1451416 ,  0.05764771, -0.05407715,  0.04977417, -0.09295654,
        0.15979004,  0.08251953,  0.03430176, -0.12548828, -0.05374146,
       -0.03567505,  0.05725098,  0.01432037,  0.09197998,  0.05490112,
       -0.02153015,  0.20117188,  0.05340576,  0.15795898,  0.06

In [3]:
import numpy as np

documents = model_f_i._preprocess_queries(queries)
encoded = model_f_i.tokenize(documents)
input_ids = np.array([[2, 9413] + e.ids[2:] for e in encoded])

attention_mask = np.array([e.attention_mask for e in encoded])
onnx_input = {"input_ids": np.array(input_ids, dtype=np.int64)}
onnx_input = model_f_i._preprocess_onnx_input(onnx_input)
onnx_input["attention_mask"] = attention_mask

In [7]:
import numpy as np
import onnxruntime as ort
from colpali_engine.models import ColPaliProcessor

model_name = "vidore/colpali-v1.2-merged"

processor = ColPaliProcessor.from_pretrained(model_name)
processor.tokenizer.max_length = 10
batch_queries = processor.process_queries(queries, max_length=5)
pixel_values = np.zeros((batch_queries["input_ids"].shape[0], 3, 448, 448), dtype=np.float32)

In [5]:
assert np.allclose(onnx_input["input_ids"], batch_queries["input_ids"])
assert np.allclose(onnx_input["attention_mask"], batch_queries["attention_mask"])
assert np.allclose(onnx_input["pixel_values"], pixel_values)

In [8]:
sess = ort.InferenceSession("/Users/d.rudenko/dev/qdrant/colpali-v1.2-merged-onnx/model.onnx")

query_embeddings = sess.run(
    [sess.get_outputs()[0].name],
    {
        "input_ids": batch_queries["input_ids"].numpy(),
        "pixel_values": pixel_values,
        "attention_mask": batch_queries["attention_mask"].numpy(),
    },
)[0]
query_embeddings = np.array(query_embeddings)

In [33]:
print(np.round(query_embeddings[0][:, :7], decimals=3))

[[ 0.158 -0.02   0.1   -0.023  0.045  0.031  0.071]
 [-0.074 -0.111  0.065 -0.    -0.089 -0.003 -0.099]
 [-0.034 -0.014  0.174 -0.063 -0.09  -0.036  0.064]
 [-0.07  -0.014  0.186 -0.013 -0.021 -0.062  0.107]
 [-0.085  0.025  0.179 -0.101  0.036 -0.089  0.098]
 [-0.058  0.031  0.18  -0.078  0.023 -0.119  0.131]
 [-0.067  0.038  0.188 -0.079 -0.001 -0.123  0.127]
 [-0.063  0.037  0.204 -0.069  0.003 -0.118  0.134]
 [-0.054  0.036  0.212 -0.072 -0.001 -0.117  0.133]
 [-0.044  0.03   0.218 -0.077 -0.003 -0.107  0.139]
 [-0.037  0.033  0.22  -0.088  0.    -0.095  0.146]
 [-0.031  0.041  0.213 -0.092  0.001 -0.088  0.147]
 [-0.026  0.047  0.204 -0.089 -0.002 -0.084  0.144]
 [-0.027  0.051  0.199 -0.084 -0.007 -0.083  0.14 ]
 [-0.031  0.056  0.19  -0.082 -0.011 -0.086  0.135]
 [-0.008  0.108  0.144 -0.095 -0.018 -0.086  0.085]]


In [8]:
query_embeddings.shape

(2, 16, 128)

In [10]:
np.allclose(query_embeddings, np.array(fastembed_q_embeddings))

True

In [11]:
np.allclose(query_embeddings, np.array(fastembed_q_embeddings))

True